In [2]:
# --- Install missing dependencies ---
!pip install -q ultralytics
!pip install -q deep_sort_realtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00


In [10]:
import warnings
warnings.filterwarnings(
    "ignore",
    message=r".*torch\.cuda\.amp\.autocast.*deprecated.*",
    category=FutureWarning
)

In [8]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

In [9]:
VIDEO_INPUT_PATH  = '/content/2863232-uhd_3840_2160_30fps.mp4'
VIDEO_OUTPUT_PATH = 'output_tracked_yolo8.mp4'


In [ ]:
CONFIDENCE_THRES = 0.3

TRACK_COLORS = [
    (255, 0, 0),     # blue-ish
    (0, 255, 0),     # green
    (0, 0, 255),     # red
    (255, 255, 0),   # cyan
    (255, 0, 255),   # magenta
    (0, 255, 255),   # yellow
]

# Load YOLOv8 model


In [ ]:
model = YOLO('yolov8s.pt')
model.fuse()
model.conf = CONFIDENCE_THRES


In [11]:


#  Open video input
cap = cv2.VideoCapture(VIDEO_INPUT_PATH)
if not cap.isOpened():
    raise IOError(f"Cannot open video: {VIDEO_INPUT_PATH}")

# Prepare video writer
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(VIDEO_OUTPUT_PATH, fourcc, fps, (width, height))

# Dictionary for saving track trajectories
trajectories = {}

# --- Processing loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # A. Use YOLOv8 tracker
    results = model.track(source=frame, persist=True, verbose=False)[0]

    # B. Process YOLOv8 tracking results
    if results and results.boxes is not None:
        boxes = results.boxes
        for box in boxes:
            cls = int(box.cls[0])
            if cls != 0:  # Only 'person' class
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            track_id = int(box.id[0]) if box.id is not None else -1
            if track_id == -1:
                continue

            cx, cy = (x1 + x2) // 2, y2

            # Save trajectory point
            trajectories.setdefault(track_id, []).append((cx, cy))

            # Draw bounding box and ID
            color = TRACK_COLORS[track_id % len(TRACK_COLORS)]
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # C. Draw trajectories
    for track_id, pts in trajectories.items():
        color = TRACK_COLORS[track_id % len(TRACK_COLORS)]
        for i in range(1, len(pts)):
            cv2.line(frame, pts[i - 1], pts[i], color, 3)

    # D. Save output frame
    out.write(frame)

# --- Cleanup ---
cap.release()
out.release()

print(f" Processing complete. Output saved to {VIDEO_OUTPUT_PATH}")


YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
 Processing complete. Output saved to output_tracked_yolo8.mp4
